# 2024 Year-End Economic Indicators Analysis
The purpose of this project is to calculate and document the performance and changes of indices, stock (singular), and economic indicators I found to be the most interesting for recapping 2024.

## Yahoo Finance
Yahoo Finance specific data retrieval and calculations.

In [1]:
import yfinance as yf

In [2]:
# dictionary of tickers the indices stock (singular)
tickers = {
    'Dow': '^DJI',
    'Nasdaq': '^IXIC',
    'S&P 500': '^GSPC',
    'TSSI': 'TSSI'
}

# set start and end dates (2024)
start_date = '2024-01-01'
end_date = '2024-12-31'

In [3]:
def calculate_return(ticker):
    """calculates performance of a given ticker"""
    data = yf.download(ticker, start=start_date, end=end_date)
    if not data.empty:
        # calc (short for calculator) performance from start to finish
        initial_price = data['Open'].iloc[0]
        final_price = data['Close'].iloc[-1]
        # return in percentage
        return (final_price - initial_price) / initial_price * 100
    else:
        return None

In [4]:
# calc and show performance
for name, ticker in tickers.items():
    performance = calculate_return(ticker)
    if performance is not None:
        print(f"{name} ({ticker}) 2024 Performance: {performance:.2f}%")
    else:
        print(f"{name} ({ticker}) data not available.")

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Dow (^DJI) 2024 Performance: 13.33%
Nasdaq (^IXIC) 2024 Performance: 31.02%
S&P 500 (^GSPC) 2024 Performance: 24.48%


[*********************100%%**********************]  1 of 1 completed

TSSI (TSSI) 2024 Performance: 4292.59%


## FRED
Federal Reserve Economic Data specific data retrieval and calculations. \
Sections are split by series IDs/specific indicator.

In [5]:
from fredapi import Fred
import pandas as pd

In [6]:
# retrive api key
# please don't leak please don't leak please don't leak
with open('/Users/erys/fred_api_key.txt', 'r') as api_file:
    api_key = api_file.readline()

In [7]:
# instantiate Fred class
fred = Fred(api_key=api_key)

In [8]:
# dictionary of FRED series IDs for desired metrics
metrics = {
    'Federal Funds Rate Target (Upper Limit)': 'DFEDTARU',
    'Headline CPI Inflation': 'CPIAUCNS',
    'Consumer Revolving Credit': 'CCLACBW027SBOG',
    'Total Nonfarm Job Openings': 'JTSJOL',
    'Unemployment Rate': 'UNRATE',
    'Job Postings on Indeed': 'IHLIDXUS'
}

i probably could've saved a couple lines and reduce repeating the same logic, but i knew i'd need to perform different actions depending on the series id, so i just copy-and-pasted :/

### Fed Funds Rate

In [9]:
fed_funds_rate = fred.get_series(metrics['Federal Funds Rate Target (Upper Limit)'], start_date, end_date)

In [10]:
fed_funds_rate_start = fed_funds_rate.iloc[0]
fed_funds_rate_end = fed_funds_rate.iloc[-1]

In [11]:
print(f"Starting Rate: {fed_funds_rate_start}%")
print(f"Final Rate: {fed_funds_rate_end}%")
print(f"Change: {fed_funds_rate_end - fed_funds_rate_start}%")

Starting Rate: 5.5%
Final Rate: 4.5%
Change: -1.0%


### CPI

In [12]:
cpi = fred.get_series(metrics['Headline CPI Inflation'], '2023-01-01', end_date)

In [13]:
cpi_jan = ((cpi.loc['2024-01-01'] - cpi.loc['2023-01-01']) / cpi.loc['2023-01-01']) * 100
cpi_nov = ((cpi.loc['2024-11-01'] - cpi.loc['2023-11-01']) / cpi.loc['2023-11-01']) * 100

In [14]:
print(f"January 2024 CPI: {cpi_jan}%")
print(f"November 2024 CPI: {cpi_nov}%")
print(f"Change: {round((cpi_nov - cpi_jan), 2)}%")

January 2024 CPI: 3.0908847812280498%
November 2024 CPI: 2.7493803960905545%
Change: -0.34%


### Consumer Revolving Credit

In [15]:
credit = fred.get_series(metrics['Consumer Revolving Credit'], start_date, end_date)

In [16]:
credit_start = credit.iloc[0] * 1e9
credit_end = credit.iloc[-1] * 1e9

In [17]:
print(f"Starting Credit Amount: ${(credit.iloc[0] * 1e9):,}")
print(f"Final Credit Amount: ${(credit.iloc[-1] * 1e9):,}")
print(f"Change: ${(credit_end - credit_start):,}")

Starting Credit Amount: $1,035,512,700,000.0
Final Credit Amount: $1,088,658,300,000.0
Change: $53,145,600,000.0


### Total Nonfarm Job Openings

In [18]:
nonfarm_jobs = fred.get_series(metrics['Total Nonfarm Job Openings'], start_date, end_date)

In [19]:
nonfarm_jobs_start = nonfarm_jobs.iloc[0] * 1e3
nonfarm_jobs_end = nonfarm_jobs.iloc[-1] * 1e3

In [20]:
print(f"Starting Job Openings: {nonfarm_jobs_start:,}")
print(f"Final Job Openings: {nonfarm_jobs_end:,}")
print(f"Change: {(nonfarm_jobs_end - nonfarm_jobs_start):,}")

Starting Job Openings: 8,748,000.0
Final Job Openings: 7,744,000.0
Change: -1,004,000.0


### Unemployment Rate

In [21]:
unemployment = fred.get_series(metrics['Unemployment Rate'], start_date, end_date)

In [22]:
unemployment_start = unemployment.iloc[0]
unemployment_end = unemployment.iloc[-1]

In [23]:
print(f"Starting Unemployment: {unemployment_start}%")
print(f"Final Unemployment: {unemployment_end}%")
print(f"Change: {(unemployment_end - unemployment_start):.2f}%")

Starting Unemployment: 3.7%
Final Unemployment: 4.2%
Change: 0.50%


### Job Postings on Indeed

In [24]:
indeed_jobs = fred.get_series(metrics['Job Postings on Indeed'], start_date, end_date)

In [25]:
indeed_jobs_start = indeed_jobs.iloc[0]
indeed_jobs_end = indeed_jobs.iloc[-1]

In [26]:
print(f"Starting Indeed Jobs: {indeed_jobs_start}")
print(f"Final Indeed Jobs: {indeed_jobs_end}")
print(f"Change: {indeed_jobs_end - indeed_jobs_start}")

Starting Indeed Jobs: 121.65
Final Indeed Jobs: 110.36
Change: -11.290000000000006
